In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from featurewiz import featurewiz
import numpy as np
import warnings
warnings.simplefilter('ignore')

Imported version = 0.1.55.
from featurewiz import FeatureWiz
wiz = FeatureWiz(verbose=1)
X_train_selected = wiz.fit_transform(X_train, y_train)
X_test_selected = wiz.transform(X_test)
wiz.features  ### provides a list of selected features ###
                                


In [2]:
testDf = pd.read_csv('./Data/test.csv')
trainDf = pd.read_csv('./Data/train.csv')
subDf = pd.read_csv('./Data/sample_submission.csv')

id = subDf.ID

In [3]:
trainDf.drop(['ID','age_desc'],axis=1,inplace=True)
testDf.drop(['ID','age_desc'],axis=1,inplace=True)

In [4]:
trainDf['ethnicity'] = trainDf['ethnicity'].replace('?','others')
trainDf['ethnicity'] = trainDf['ethnicity'].replace('Others','others')
trainDf['relation'] = trainDf['relation'].replace('?','Others')

In [5]:
testDf['ethnicity'] = testDf['ethnicity'].replace('?','others')
testDf['ethnicity'] = testDf['ethnicity'].replace('Others','others')
testDf['relation'] = testDf['relation'].replace('?','Others')

In [6]:
trainDf['gender'] = np.where(trainDf['gender']=='f',0,1)
trainDf['jaundice'] = np.where(trainDf['jaundice']=='no',0,1)
trainDf['austim'] = np.where(trainDf['austim']=='no',0,1)
trainDf['used_app_before'] = np.where(trainDf['used_app_before']=='no',0,1)

In [7]:
testDf['gender'] = np.where(testDf['gender']=='f',0,1)
testDf['jaundice'] = np.where(testDf['jaundice']=='no',0,1)
testDf['austim'] = np.where(testDf['austim']=='no',0,1)
testDf['used_app_before'] = np.where(testDf['used_app_before']=='no',0,1)

In [8]:
from sklearn.preprocessing import LabelEncoder

#Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df

In [9]:
trainDf = dummyEncode(trainDf)
testDf = dummyEncode(testDf)

In [10]:
outliers_values = []
for cols in trainDf.columns[:-1]:
    high = trainDf[cols].mean() + 1.5*trainDf[cols].std()
    low = trainDf[cols].mean() - 1.5*trainDf[cols].std()
    print(f"Highest allowed for {cols} is {high} and Lowest allowed is {low}")
    outliers_values.append([low,high,cols])

Highest allowed for A1_Score is 1.3050462182537186 and Lowest allowed is -0.18504621825371848
Highest allowed for A2_Score is 1.2791171274161572 and Lowest allowed is -0.2191171274161572
Highest allowed for A3_Score is 1.1967074158974154 and Lowest allowed is -0.2967074158974153
Highest allowed for A4_Score is 1.1545454071668466 and Lowest allowed is -0.3245454071668466
Highest allowed for A5_Score is 1.1287347670551546 and Lowest allowed is -0.3387347670551545
Highest allowed for A6_Score is 0.9939955418255249 and Lowest allowed is -0.38649554182552487
Highest allowed for A7_Score is 1.132030705073566 and Lowest allowed is -0.337030705073566
Highest allowed for A8_Score is 1.2591042655171119 and Lowest allowed is -0.24160426551711178
Highest allowed for A9_Score is 1.245431665513761 and Lowest allowed is -0.25543166551376106
Highest allowed for A10_Score is 1.3469525781911988 and Lowest allowed is -0.11195257819119886
Highest allowed for age is 52.91856646502906 and Lowest allowed is 

In [11]:
trainDf.shape

(800, 20)

In [12]:
df = pd.DataFrame([],columns = trainDf.columns)
for vals in outliers_values:
    low = vals[0]
    high = vals[1]
    cols = vals[-1]
    df = df.append(trainDf[(trainDf[cols] > high) | (trainDf[cols] < low)], ignore_index=True)

In [15]:
df.shape

(1051, 20)

In [17]:
out_df = df.copy()

In [18]:
for cols in out_df.columns:
    upper_limit = df[cols].mean() + 1.5*df[cols].std()
    lower_limit = df[cols].mean() - 1.5*df[cols].std()
    out_df[cols] = np.where(out_df[cols]>upper_limit,upper_limit,np.where(out_df[cols]<lower_limit,lower_limit,out_df[cols]))

In [20]:
out_df['Class/ASD'].value_counts()

0    723
1    328
Name: Class/ASD, dtype: int64

In [50]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from imblearn.over_sampling import SMOTE

In [46]:
oversample = SMOTE()
X_new, y_new = oversample.fit_resample(trainDf.drop('Class/ASD',axis=1),trainDf['Class/ASD'])

In [ ]:
X,y = trainDf.drop('Class/ASD',axis=1),trainDf['Class/ASD']

In [28]:
y.shape

(800,)

In [51]:
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X_new.drop('result',axis=1),y_new)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_new.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 14 best features

            Specs       Score
15  contry_of_res  723.781552
10            age  214.636379
5        A6_Score  208.564460
3        A4_Score  187.501340
8        A9_Score  152.883749
2        A3_Score  150.520833
6        A7_Score  141.453333
4        A5_Score  134.260274
1        A2_Score   83.712195
9       A10_Score   70.812766


In [52]:
feat_top = pd.DataFrame(featureScores.nlargest(10,'Score'))
cols = feat_top.Specs
cols = cols.tolist()
X_new = X_new[cols]

In [54]:
count_class_1 = y_new.value_counts()[0]
count_class_2 = y_new.value_counts()[1]
ratio = count_class_1/count_class_2
ratio

1.0

In [55]:
from sklearn.preprocessing import StandardScaler

In [56]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

In [57]:
X_scaled

array([[-1.70205477,  0.50743831, -0.9029623 , ...,  0.97221679,
        -1.33805568,  0.59964529],
       [-0.6751167 ,  1.0872158 , -0.9029623 , ..., -1.02857718,
        -1.33805568, -1.66765257],
       [ 0.97939465, -1.35653667,  1.10746595, ...,  0.97221679,
         0.74735305,  0.59964529],
       ...,
       [-1.816159  ,  1.09283559,  1.10746595, ...,  0.97221679,
         0.74735305,  0.59964529],
       [ 0.8082383 ,  2.22312339,  1.10746595, ..., -1.02857718,
         0.74735305,  0.59964529],
       [-1.53089843, -1.06467849,  1.10746595, ...,  0.97221679,
         0.74735305,  0.59964529]])

In [58]:
X_test, X_train, y_test, y_train = train_test_split(X_scaled, y_new, test_size=0.80, random_state=1,shuffle=True)

In [59]:
rf = RandomForestClassifier(class_weight={1:ratio, 0:1},n_estimators=10,criterion="gini",max_depth=5, min_samples_split=4)

In [60]:
# rf_model = rf.fit(X_scaled,y_new)
# rf_pred = rf_model.predict(X_test)
# print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       0.94      0.83      0.88       122
           1       0.86      0.95      0.90       133

    accuracy                           0.89       255
   macro avg       0.90      0.89      0.89       255
weighted avg       0.90      0.89      0.89       255



In [39]:
rf_model = rf.fit(X_scaled,y)
rf_pred = rf_model.predict(X_test)
print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       0.99      0.88      0.93       125
           1       0.69      0.97      0.81        35

    accuracy                           0.90       160
   macro avg       0.84      0.93      0.87       160
weighted avg       0.93      0.90      0.91       160



In [43]:
testDf

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jaundice,austim,contry_of_res,used_app_before,result,relation
0,1,1,0,0,1,1,0,0,1,1,15.599481,1,8,1,0,14,0,12.399055,4
1,1,0,0,0,0,0,0,1,0,0,27.181099,1,0,0,0,21,0,6.551598,4
2,1,1,1,0,1,1,0,1,1,1,31.643906,1,8,1,0,10,0,3.180663,4
3,0,0,0,0,0,0,0,0,0,0,25.369210,1,9,0,0,14,0,2.220766,4
4,0,0,0,1,0,0,0,0,0,0,9.078580,1,9,0,0,17,0,7.252028,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,1,0,0,1,0,0,1,1,1,23.099434,1,1,0,0,4,0,-1.915659,4
196,1,0,0,0,0,0,0,0,0,1,13.935726,1,9,0,0,14,0,0.520234,4
197,1,0,0,0,0,0,1,0,1,1,22.760041,1,9,0,0,23,0,3.498948,1
198,0,1,0,0,0,0,0,1,0,1,24.352584,0,9,0,0,32,0,5.594550,4


In [63]:
rf.fit(X_scaled,y)
rf_y_pred = rf_model.predict(testDf[cols])

ValueError: Found input variables with inconsistent numbers of samples: [1278, 800]

In [62]:
su = pd.DataFrame({'ID':id, 'Class/ASD':rf_y_pred})
su['Class/ASD'].value_counts()

1    200
Name: Class/ASD, dtype: int64

In [44]:
su.to_csv("rf_sub.csv",index=False)

In [45]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [46]:
rf_random = RandomizedSearchCV(estimator = rf_model, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [47]:
rf_hyper = rf_random.fit(X_scaled, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been depreca

[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   1.1s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   1.1s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   1.1s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.6s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.6s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.6s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   1.7s
[CV] END bootstrap=False, max_depth=30, m

/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=   4.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time=   5.2s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time=   5.2s


...

[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   1.6s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   1.6s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   1.6s


/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been depreca

[CV] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   1.1s
[CV] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.6s
[CV] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   1.2s
[CV] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.6s
[CV] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.6s


/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/aneruthmohanasundaram/Documents/GitHub/Autism-Prediction/.venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been depreca

[CV] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   1.2s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=   5.4s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=   5.5s
[CV] END bootstrap=False, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=2000; total time=   5.4s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   2.8s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   2.8s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   2.8s
[CV] END bootstrap=True, max

In [48]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [49]:
model_rf = RandomForestClassifier(class_weight={1:ratio, 0:1},n_estimators=400,bootstrap= True,max_depth= 30,max_features='sqrt',min_samples_leaf=1,min_samples_split=5)

In [50]:
rf_mdl = model_rf.fit(X_scaled,y)
rf_grid_pred = rf_mdl.predict(X_test)
print(classification_report(y_test, rf_grid_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97       129
           1       0.82      1.00      0.90        31

    accuracy                           0.96       160
   macro avg       0.91      0.97      0.94       160
weighted avg       0.96      0.96      0.96       160



In [51]:
model_rf.fit(X_scaled,y)
rf_y_pred = rf_mdl.predict(testDf[cols])

In [52]:
su = pd.DataFrame({'ID':id, 'Class/ASD':rf_y_pred})
su['Class/ASD'].value_counts()

0    140
1     60
Name: Class/ASD, dtype: int64

In [53]:
su.to_csv("rf_grid_sub2.csv",index=False)

In [56]:
from xgboost import XGBClassifier

In [58]:
xgb_model = XGBClassifier().fit(X_train, y_train)

In [59]:
xgb_y_predict = xgb_model.predict(X_test)

In [60]:
xgb_score = accuracy_score(xgb_y_predict, y_test)

In [64]:
xgb_model.fit(X_scaled,y)
xgb_y_pred = xgb_model.predict(testDf[cols])

In [65]:
su = pd.DataFrame({'ID':id, 'Class/ASD':xgb_y_pred})
su['Class/ASD'].value_counts()

0    167
1     33
Name: Class/ASD, dtype: int64